<a href="https://colab.research.google.com/github/AidanJIC/Clasificaci-n-de-productos-en-supermercados-utilizando-visi-n-computacional/blob/main/Laboratorios/Pr%C3%A1ctica_PDSeI_01_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch

## Instalaciones

In [ ]:
!pip install torch
!pip install torchvision

## Importar dependencias

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

## Definir Hiperparámetros

In [ ]:
input_size = 784
hidden_size = 128
num_classes = 10
num_epochs = 5
batch_size = 100
lr = 1e-3

## Descargando la base de datos mnist

In [ ]:
train_data = dsets.FashionMNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = dsets.FashionMNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

100%|██████████| 26421880/26421880 [00:11<00:00, 2237979.91it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 202442.63it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3725074.99it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 24179481.51it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



## Leyendo la data

In [ ]:
train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size,
                                      shuffle = False)

## Definir modelo

In [ ]:
class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.drop = nn.Dropout(0.2)
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.drop(out)
    out = self.fc2(out)
    return out

## Instancia del modelo

In [ ]:
net = Net(input_size, hidden_size, num_classes)

if torch.cuda.is_available():
  net.cuda()

## Compilación

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

## Entrenamiento

In [ ]:
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_gen):
    images = images.view(-1, 28*28).cuda()
    labels = labels.cuda()

    optimizer.zero_grad()
    outputs = net(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    if (i + 1) % 100 == 0:
        print('Epoca [%d/%d], Step [%d/%d], Loss: %.4f'
              % (epoch + 1, num_epochs, i + 1, len(train_data) // batch_size, loss.item()))


Epoca [1/5], Step [100/600], Loss: 0.6858
Epoca [1/5], Step [200/600], Loss: 0.4623
Epoca [1/5], Step [300/600], Loss: 0.5077
Epoca [1/5], Step [400/600], Loss: 0.5390
Epoca [1/5], Step [500/600], Loss: 0.4710
Epoca [1/5], Step [600/600], Loss: 0.4209
Epoca [2/5], Step [100/600], Loss: 0.4241
Epoca [2/5], Step [200/600], Loss: 0.3720
Epoca [2/5], Step [300/600], Loss: 0.3053
Epoca [2/5], Step [400/600], Loss: 0.2724
Epoca [2/5], Step [500/600], Loss: 0.4052
Epoca [2/5], Step [600/600], Loss: 0.5020
Epoca [3/5], Step [100/600], Loss: 0.2904
Epoca [3/5], Step [200/600], Loss: 0.4464
Epoca [3/5], Step [300/600], Loss: 0.2642
Epoca [3/5], Step [400/600], Loss: 0.4933
Epoca [3/5], Step [500/600], Loss: 0.4052
Epoca [3/5], Step [600/600], Loss: 0.3526
Epoca [4/5], Step [100/600], Loss: 0.4071
Epoca [4/5], Step [200/600], Loss: 0.3768
Epoca [4/5], Step [300/600], Loss: 0.3226
Epoca [4/5], Step [400/600], Loss: 0.2910
Epoca [4/5], Step [500/600], Loss: 0.3905
Epoca [4/5], Step [600/600], Loss:

In [ ]:
correct = 0
total = 0
for images, labels in test_gen:
    images = images.view(-1, 28*28).cuda()
    labels = labels.cuda()

    output = net(images)
    _, predicted = torch.max(output, 1)
    correct += (predicted == labels).sum()
    total += labels.size(0)

print('Accuracy: %.3f %%' % (100 * correct / (total + 1)))

Accuracy: 86.041 %
